In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers
!pip install -U bitsandbytes
!pip install accelerate
!pip install together

In [ ]:
import transformers
import torch
from kaggle_secrets import UserSecretsClient
from together import Together
user_secrets = UserSecretsClient()
your_token = user_secrets.get_secret("HuggingFaceSecret")
tokenizer_gemma = transformers.AutoTokenizer.from_pretrained("google/gemma-2b-it",use_auth_token=your_token)
# model_gemma = transformers.AutoModelForCausalLM.from_pretrained(
#     "google/gemma-2b-it",
#     torch_dtype=torch.bfloat16,
#     use_auth_token=your_token,
#     load_in_4bit=True
# )

# gen_config_gemma = transformers.GenerationConfig.from_pretrained(
#     "google/gemma-2b-it"
# )
# gen_config_gemma.max_new_tokens = 500
# gen_config_gemma.temperature = 1e-10
 
# pipeline_gemma = transformers.pipeline(
#     task="text-generation",
#     model=model_gemma,
#     tokenizer=tokenizer_gemma,
#     return_full_text=True,
#     generation_config=gen_config_gemma,
#     device_map='auto',
#     repetition_penalty=1.1,
#     eos_token_id=tokenizer_gemma.eos_token_id,
#     pad_token_id=tokenizer_gemma.pad_token_id
# ) 
# torch.random.manual_seed(0)


model_microsoft = transformers.AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct", 
    device_map="cuda", 
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    trust_remote_code=True, 
    use_auth_token=your_token,
    
)
tokenizer_microsoft = transformers.AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
gen_config_microsoft = transformers.GenerationConfig.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct"
)
gen_config_microsoft.max_new_tokens = 400
 
pipeline_microsoft = transformers.pipeline(
    task="text-generation",
    model=model_microsoft,
    tokenizer=tokenizer_microsoft,
    generation_config=gen_config_microsoft,
    device_map='cuda',
    eos_token_id=tokenizer_microsoft.eos_token_id,
    pad_token_id=tokenizer_microsoft.pad_token_id
)        

# client = Together(api_key="e52e5c120584a8119f7e30c8e2141f14310f9830af01ba05a06bdee51faa93c8")
# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# tokenizer_llama = transformers.AutoTokenizer.from_pretrained(model_id,use_auth_token=your_token)
# model_llama = transformers.AutoModelForCausalLM.from_pretrained(
#     model_id,
#     use_auth_token=your_token,
#     device_map="cuda",
#     load_in_4bit=True
# )

# gen_config_llama = transformers.GenerationConfig.from_pretrained(
#     model_id
# )
# gen_config_llama.max_new_tokens = 500
# gen_config_llama.temperature = 1e-10
 
# pipeline_llama = transformers.pipeline(
#     task="text-generation",
#     model=model_llama,
#     tokenizer=tokenizer_llama,
#     return_full_text=True,
#     generation_config=gen_config_llama,
#     device_map='auto',
#     model_kwargs={
#         'load_in_4bit': True,
#         'bnb_4bit_quant_type': 'nf4',
#         'bnb_4bit_use_double_quant': True,
#         'bnb_4bit_compute_dtype': torch.bfloat16,
#     },
#     repetition_penalty=1.1,
#     eos_token_id=tokenizer_llama.eos_token_id,
#     pad_token_id=tokenizer_llama.pad_token_id
# )        

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain_huggingface
!pip install datasets
!pip install Levenshtein

In [ ]:
def zero_shot_pipeline(pipeline, prompt):
    tokenizer=pipeline.tokenizer
    start_time = time.time()  # Start time

   

    generated_text = pipeline(prompt)

    
    end_time = time.time()  # End time
    time_taken = end_time - start_time
    output_texts=[output['generated_text'] for output in generated_text]
    formatted_output="".join(output_texts)
    return formatted_output, time_taken

# Function for Chain-of-Thought Prompting
def chain_of_thought_pipeline(pipeline, prompt):
    tokenizer=pipeline.tokenizer
    
    
    start_time = time.time()  # Start time

    
    generated_text = pipeline(prompt)

    end_time = time.time()  # End time
    time_taken = end_time - start_time

    output_texts=[output['generated_text'] for output in generated_text]
    formatted_output="".join(output_texts)
    return formatted_output, time_taken


In [ ]:
from langchain import hub
prompt_react = hub.pull("hwchase17/react")
print(prompt_react.template)

In [ ]:

from datasets import load_dataset

# Load the MMLU test dataset from Hugging Face
dataset = load_dataset("cais/mmlu", 'college_mathematics',split="test")

question_answer_dict = {row['question']: row['choices'] for row in dataset}

# Display the first few entries of the dictionary to confirm
first_entries = dict(list(question_answer_dict.items())[:5])
print(first_entries)

answers=[row['answer'] for row in dataset]
print(len(answers))


In [ ]:
import re
# def parse_output(text):
#     match = re.search(r'Answer\s*::\s*(\d)', text)
    
#     if match:
#         return match.group(1)  # Extract the first capturing group (the digit)
#     else:
#         return None
    
def parse_output(text):
    # Regex pattern to match variations like 'answer is _', 'answer: _', or 'final answer:_'
    pattern = r'(?i)(?:answer\s*is|answer\s*:|final\s*answer\s*:|answer\s*:\s*option|final\s*answer\s*:\s*option|final\s*answer\s*is\s*:?\s*\\boxed\{|\boxed\{|answer\s*:\s*\{)(\d)'
    match = re.search(pattern, text)
    
    if match:
        return match.group(1)  # Extract the matched digit
    else:
        return None  # Return None if no match is found

    
def calculate_accuracy(outputs,answers=answers,num_questions=100):
    num_correct=0
    for i in range(num_questions):
        if(outputs[i] is not None and (int(outputs[i])==int(answers[i]+1))):
            num_correct+=1
    print(num_correct)
    accuracy=(num_correct/num_questions)*100
    return accuracy

def zero_shot_prompt(question,options):
    return f"""Please select the correct answer by responding *only* with the corresponding number (1, 2, 3, or 4). No explanation needed. Stick to given format.


        Question: {question}

        Options:
        1) {options[0]}
        2) {options[1]}
        3) {options[2]}
        4) {options[3]}

        Answer :"""

def chain_of_thought_prompt(question,options):
    return "Choose the answer of the given question from the below options.\n"+question+"\n"+"{1:"+options[0]+"}"+"\n"+"{2:"+options[1]+"}"+"\n"+"{3:"+options[2]+"}"+"\n"+"{4:"+options[3]+"}"+"\n"+"Think step by step."+"\n"+"answer: {"

In [ ]:
from Levenshtein import distance
def parse_output_ReAct(text):
    text = str(text).strip()  # Ensure the input is a string and remove any leading/trailing whitespace
    lower_text = text.lower()  # Convert to lowercase for case-insensitive matching

    # Check for the last occurrence of each pattern using rfind()

    # Check for cases like "The final answer is: $\boxed{3}$" or "The answer is"
    if lower_text.rfind("the final answer is:") != -1 or lower_text.rfind("the answer is") != -1:
        
        boxed_start = lower_text.rfind("\\boxed{")
        if boxed_start != -1:
            digit_start = boxed_start + len("\\boxed{")
            digit_end = lower_text.find("}", digit_start)
            if digit_end != -1:
                return text[digit_start:digit_end]  # Extract the digit from the original text

    # Check for the last occurrence of formats like "answer is 3" or "answer: 2"
    if lower_text.rfind("answer is") != -1:
        
        start_pos = lower_text.rfind("answer is") + len("answer is")
        return extract_digit(text[start_pos:])  # Use the original text to extract the digit
    
    if lower_text.rfind("answer:") != -1:
        
        start_pos = lower_text.rfind("answer:") + len("answer:")
        return extract_digit(text[start_pos:])

    # Check for "final answer: option 4" or "answer: option 4"
    if lower_text.rfind("final answer: option") != -1 or lower_text.rfind("answer: option") != -1:
        
        start_pos = lower_text.rfind("option") + len("option")
        return extract_digit(text[start_pos:])
    
    # Check for the last occurrence of "answer: {2:n = 1 and r = 7}"
    if lower_text.rfind("answer: {") != -1:
        
        start_pos = lower_text.rfind("answer: {") + len("answer: {")
        return extract_digit(text[start_pos:])
    
    # If no patterns match, return None
    return None

def extract_digit(subtext):
    # Extract the first digit found in the substring
    for char in subtext.strip():
        if char.isdigit():
            return char
    return None
def find_most_similar_option(options, target):

    min_distance = float('inf')
    best_match = None
    best_index = -1
    
    for index, option in enumerate(options):
        dist = distance(target, option)
        if dist < min_distance:
            min_distance = dist
            best_match = option
            best_index = index
            
    return best_index+1


In [ ]:
import time

pipeline=pipeline_microsoft
time_zero=0
outputs_zero_shot=[]
num_questions=0
for question,options in question_answer_dict.items():
    print("Zero-SHOT")
    prompt=zero_shot_prompt(question,options)
    Zero_shot_output,zero_shot_time=zero_shot_pipeline(pipeline,prompt)
    option=parse_output(Zero_shot_output)
    if(option is not None):
        num_questions+=1
    print(Zero_shot_output)
    outputs_zero_shot.append(option)
    time_zero+=zero_shot_time
    print("----------------------------------------------------")
acc=calculate_accuracy(outputs_zero_shot,answers,num_questions)
print(f"Accuracy:{acc}")
print(f"Time:{time_zero/100}")
    

In [ ]:
import time
pipeline=pipeline_microsoft
time_CoT=0
outputs_CoT=[]
num_questions=0
for question,options in question_answer_dict.items():
    print("Chain of Thought")
    prompt=chain_of_thought_prompt(question,options)
    CoT_output,CoT_time=chain_of_thought_pipeline(pipeline,prompt)
    option=parse_output(CoT_output)
    if(option is not None):
        num_questions+=1
    print(CoT_output)
    outputs_CoT.append(option)
    time_CoT+=CoT_time
    print("----------------------------------------------------")
acc=calculate_accuracy(outputs_CoT,answers,num_questions)
print(f"Accuracy:{acc}")
print(f"Time:{time_CoT/100}")
    

In [ ]:
from langchain import hub
prompt_react = hub.pull("hwchase17/react")
print(prompt_react.template)

from langchain.agents import load_tools
from langchain.agents import AgentExecutor,create_react_agent,initialize_agent,AgentType,Tool
from langchain_huggingface import HuggingFacePipeline
from io import StringIO
from transformers import TextIteratorStreamer
import sys
import re
import io

def run_and_capture(prompt, react_agent):
    # Create a string buffer to capture the output
    captured_output = io.StringIO()
    
    # Save the original stdout
    original_stdout = sys.stdout
    
    try:
        # Redirect stdout to both the buffer and the console
        sys.stdout = captured_output

        # Run the agent with the provided prompt
        result = react_agent.run(prompt)
        
        # Write the captured output directly to the original stdout (console)
        original_stdout.write(captured_output.getvalue())

    finally:
        # Reset stdout to its original state
        sys.stdout = original_stdout
    
    # Get the captured verbose output
    verbose_output = captured_output.getvalue()
    
    # Return both the final result and the verbose output
    return verbose_output

def ReAct_pipeline(pipeline,prompt):
    tokenizer=pipeline.tokenizer
    prompt_react = hub.pull("hwchase17/react")
    
    llm = HuggingFacePipeline(pipeline=pipeline)

    tools = load_tools(["llm-math"], llm=llm)


    react_agent = initialize_agent(
        llm=llm,  # The language model pipeline
        tools=tools, # The tools, e.g., llm-math  
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        prompt=prompt_react,
        handle_parsing_errors=True,
        max_iterations=1,
        verbose=True
    )
    start_time = time.time()  # Start time
    output = run_and_capture(prompt,react_agent)
    end_time = time.time()  # End time
    time_taken = end_time - start_time
    return output,time_taken



In [ ]:
pipeline=pipeline_microsoft
time_ReAct=0
outputs_ReAct=[]
num_questions=0
for question,options in question_answer_dict.items():
    print("ReAct Prompting")
    prompt=chain_of_thought_prompt(question,options)
    ReAct_output,ReAct_time=ReAct_pipeline(pipeline,prompt)
    option=parse_output_ReAct(ReAct_output)
    if(option not in ['1','2','3','4']):
        for i in range(4):
            if(option==options[i]):
                option=i+1
    if(option not in ['1','2','3','4']):
        option=find_most_similar_option(ReAct_output,options)
    if(option is not None):
        num_questions+=1
    outputs_ReAct.append(option)
    print(option)
    time_ReAct+=ReAct_time
    print("----------------------------------------------------")
acc=calculate_accuracy(outputs_ReAct,answers,num_questions)
print(f"Accuracy:{acc}")
print(f"Time:{time_ReAct/100}")
    